Updated notebook - main changes: revert to separating each category into its own collection, implement 2 step pipeline (categorize, then retrieve from those relevant collections). Somehow, takes a substantial amount of time now (ie 7 queries in 17 minutes) (using general qa set and some other random set of data), to discuss.

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
!pip install llama-cpp-python

Define Variables

In [89]:
from llama_cpp import Llama
from langchain_community.llms import HuggingFaceEndpoint
import os
from transformers import pipeline

model_path = "mistral-7b-instruct-v0.2.Q4_0.gguf"
model = Llama(model_path=model_path, n_ctx=2048, n_threads=8, verbose=False)

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_b

In [90]:
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")


Define Data Sources

In [91]:
import pandas as pd
import concurrent.futures
import uuid
import os

file_names = [
    "study_permit_general", "work_permit_student_general", "work-study-data-llm",
    "vancouver_transit_qa_pairs", "permanent_residence_student_general", "data-with-sources",
    "faq_qa_pairs_general", "hikes_qa", "sfu-faq-with-sources", "sfu-housing-with-sources",
    "sfu-immigration-faq", "park_qa_pairs-up", "cultural_space_qa_pairs_up",
    "qa_pairs_food", "qa_pairs_year_and_month_avg", "qa_pairs_sfu_clubs"
]

collections = {}
batch_size = 32

def process_file(file):
    try:
        path = f'../Data/{file}.csv'
        if not os.path.exists(path):
            return f"{file} skipped (file not found)."

        df = pd.read_csv(path, usecols=lambda col: col.lower() in {"question", "answer"})
        df.columns = df.columns.str.lower()

        if "question" not in df.columns or "answer" not in df.columns:
            return f"{file} skipped (missing question/answer columns)."

        df = df.drop_duplicates(subset="question")
        df["text"] = df["question"].fillna('') + ' ' + df["answer"].fillna('')
        unique_texts = list(set(df["text"].dropna().tolist()))

        collection = client.get_or_create_collection(name=file)
        for i in range(0, len(unique_texts), batch_size):
            batch = unique_texts[i:i + batch_size]
            embeddings = embedding_model.embed_documents(batch)
            ids = [str(uuid.uuid4()) for _ in batch]
            collection.add(ids=ids, embeddings=embeddings, documents=batch)

        collections[file] = collection
        return f"{file}: Loaded {len(unique_texts)} docs."
    except Exception as e:
        return f"{file}: Error - {e}"

# parallelogram
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    results = list(executor.map(process_file, file_names))

for result in results:
    print(result)


study_permit_general: Loaded 14 docs.
work_permit_student_general: Loaded 23 docs.
work-study-data-llm: Loaded 178 docs.
vancouver_transit_qa_pairs: Loaded 37 docs.
permanent_residence_student_general: Loaded 6 docs.
data-with-sources: Loaded 75 docs.
faq_qa_pairs_general: Loaded 220 docs.
hikes_qa: Loaded 278 docs.
sfu-faq-with-sources: Loaded 102 docs.
sfu-housing-with-sources: Loaded 74 docs.
sfu-immigration-faq: Loaded 83 docs.
park_qa_pairs-up: Loaded 216 docs.
cultural_space_qa_pairs_up: Loaded 560 docs.
qa_pairs_food: Loaded 38 docs.
qa_pairs_year_and_month_avg: Loaded 1488 docs.
qa_pairs_sfu_clubs: Loaded 166 docs.


In [94]:
# define cat to collection mapping
# motivation: takes wayyyy too long now -> dataset size trippled and time grew exponentially...
# now takes around 2 mins on avg for each response..compared to 40 seconds previously..

collection_map = {
    "study": "study_permit_general",
    "student work": "work_permit_student_general",
    "work-study": "work-study-data-llm",
    "transit": "vancouver_transit_qa_pairs",
    "permanent residence": "permanent_residence_student_general",
    "general info": "data-with-sources",
    "faq": "faq_qa_pairs_general",
    "hiking": "hikes_qa",
    "sfu faq": "sfu-faq-with-sources",
    "housing": "sfu-housing-with-sources",
    "immigration": "sfu-immigration-faq",
    "parks": "park_qa_pairs-up",
    "culture": "cultural_space_qa_pairs_up",
    "food": "qa_pairs_food",
    "weather": "qa_pairs_year_and_month_avg",
    "clubs": "qa_pairs_sfu_clubs"
}


Function to now match for releveant document

In [95]:
def get_relevant_documents(query, categories, n_results=3):
    all_results = []
    query_embedding = embedding_model.embed_documents([query])[0]

    for category in categories:
        collection_name = collection_map[category]
        if collection_name in collections:
            try:
                result = collections[collection_name].query(
                    query_embeddings=[query_embedding],
                    n_results=n_results
                )
                docs = result.get("documents", [[]])[0]
                sims = result.get("distances", [[]])[0]

                all_results.extend(zip(docs, sims))
            except Exception as e:
                print(f"error querying {collection_name}: {e}")

    all_results = sorted(all_results, key=lambda x: x[1])

    return all_results[:n_results]


### Classify Prompt

In [96]:
import re
import difflib

valid_categories = list(collection_map.keys())
fallback_category = "faq"

def classify_query(query):
    category_prompt = f"""
    You are a classifier for a Q&A system for international students in British Columbia.
    Choose the **1 most relevant** category from this list, or at most 3 if absolutely needed (comma-separated):

    {", ".join(valid_categories)}

    Query: "{query}"

    Return only the category name(s) as a comma-separated string.
    """

    response = model(category_prompt, max_tokens=50, temperature=0.1)["choices"][0]["text"].strip().lower()
    print("Raw out:", response)

    tokens = re.findall(r'\b\w+\b', response)

    matched = []
    for token in tokens:
        closest = difflib.get_close_matches(token, valid_categories, n=1, cutoff=0.8)
        if closest and closest[0] not in matched:
            matched.append(closest[0])
        if len(matched) == 3:
            break

    if fallback_category not in matched:
      matched.append(fallback_category)

    return matched[:3]


Generate Answer

In [178]:
def generate_answer(query):
    categories = classify_query(query)
    print(f"Categories {categories}\n")
    relevant_documents = get_relevant_documents(query, categories)

    if not relevant_documents:
        return {
            "Response": "Sorry, no relevant documents found."
        }

    #relevant_documents = list(set(relevant_documents))

    seen = set()
    unique_docs = []
    for doc, sim in relevant_documents:
        if doc not in seen:
            seen.add(doc)
            unique_docs.append((doc, sim))

    print("Relevant Documents with Similarity Scores:")
    for doc, sim in unique_docs:
        print(f"Similarity: {sim:.4f}\nDoc: {doc}\n")

    relevant_texts = "\n\n".join([doc for doc, _ in unique_docs])

    rag_prompt = f"""
    You are a helpful, friendly assistant for international students new to British Columbia, Canada.

    Below are some reference documents that may be relevant to the user's question:
    {relevant_texts}

    INSTRUCTIONS:
    1. If the user's query is just a greeting (like "hello", "hi", "what's up"):
       - Respond with a single brief friendly greeting
       - Offer to help with questions about studying or living in BC
       - Do NOT include ANY information from the reference documents
       - Do NOT create additional questions and answers

    2. If the user is asking for information:
       - Be friendly and answer based ONLY on the reference documents if relevant
       - Keep your answer brief and concise (under 30 words when possible)
       - Summarize the necessary information into a couple sentences.
       - Limit your entire response to no more than 3 concise sentences when possible. Do not create long multi-line answers.
       - If the documents don't provide sufficient information, say "I don't have enough information to answer that. Please refer to official sources."
       - Do NOT create additional questions and answers beyond answering their original question
       - Ask for more information when there are multiple senarios in the documents.
       - If they ask things like "can I", "will I", "how can I" feel free to ask follow up questions if you don't how to answer with the information provided. Do not just assume.
       - If they asked only asked a question and did not provide information surrounding it, there is no need to state "Based on the information provided,"
       - Always answer in full sentences.
    
    3. For hiking information specifically:
       - Convert structured information about the hike into a short, friendly paragraph using natural language. Do not repeat numbers or use formatting from the source.
       - If they ask about hiking information, only answer with required information. Users can ask for more information if needed.
       - Use only the most relevant details (like difficulty, time, and unique traits).
       - When asked for a particular type of hike, find it instead of saying that one would not work in the category they asked for.
       - If they ask for suggestions, provide 2 to 3 suggestions.
       - Do NOT list trail attributes or stats (like “Distance: 3.1 km, Elevation: 789 m”). Instead, describe them in context (e.g., “a steep 3 km trail with a tough 789 m climb”).
       - Avoid repeating exact numbers unless essential (e.g., elevation gain is helpful, but don’t dump all stats).
       
    4. IMPORTANT: Never generate additional content beyond answering the user's question
    
    5. IMPORTANT: Do NOT number or bullet your points. Always use natural sentences and group similar information together where possible.

    User question: {query}

    Your response (just the answer, no preamble):
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]
    # response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)

    return {
        "Response": response_after_rag
    }


Example Usage

In [179]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

In [180]:
answer = generate_answer("I want to start hiking in metro vancouver.")
print(answer["Response"])

Raw out: category: hiking
Categories ['hiking', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 0.8784
Doc: Can you tell me about the BCMC Trail hike? Ranking: ☆☆☆
Difficulty: Difficult
Distance: 3.1 km
Elevation Gain: 789 m
Gradient: 0.2545
Time: 0 – 2 hours
Dogs Allowed: No
4x4 Needed: No
Season: Year-Round
Region: Metro Vancouver

1. BCMC Trail is a popular hike in Metro Vancouver. It's a challenging 3.1 km trail with a steep 789 m climb.
    2. The trail is located in the North Shore Mountains and offers stunning views of the city and surrounding nature.
    3. Hiking BCMC Trail usually takes between 0 to 2 hours, depending on your fitness level and pace.
    4. Remember to check the trail conditions before you go, as they can change due to weather.
    5. For more hiking suggestions, consider checking out the Lynn Canyon Suspension Bridge Trail or the Quarry Rock Trail.
    6. Enjoy your hiking experience in Metro Vancouver!


In [ ]:
!pip install evaluate
!pip install bert_score

In [181]:
import pandas as pd
from evaluate import load
bertscore = load("bertscore")

benchmark_data = pd.read_csv("../s-eval-set/s_test_qa.csv")

for idx, row in benchmark_data.iterrows():
    user_query = row["Questions"]
    correct_answer = row["Answers"]

    responses = generate_answer(user_query)
    
    predictions = [responses.get("Response", "N/A")]
    references = [correct_answer]
    results = bertscore.compute(predictions=predictions, references=references, lang="en")

    print("\n" + "="*50)
    print(f"Benchmark Query {idx + 1}: {user_query}")
    print("="*50)
    print("\nRAG Response:\n", responses.get("Response", "N/A"))
    print("\n(Benchmark) Answer:\n", correct_answer)
    print("BERT Score == ", results)
    print("="*50 + "\n\n")


Raw out: immigration
Categories ['immigration', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 0.6393
Doc: What is required to apply for a study permit extension? You can only extend a study permit if you are physically inside Canada at the time of application.



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Benchmark Query 1: As a minor how can I apply for a study permit in Canada?

RAG Response:
 1. You cannot apply for a study permit as a minor without a guardian in Canada.
    2. Your guardian must apply on your behalf.
    3. They will need to provide proof of their relationship to you and their ability to support you financially.
    4. You will also need to provide all required documents, including your acceptance letter from a designated learning institution.
    5. For more information, refer to the official Immigration, Refugees and Citizenship Canada website.

(Benchmark) Answer:
 Your parents will need to help with your study permit application. Both parents will need to submit documents, but if you only have one parent for any reason there are other ways of applying for a permit.
BERT Score ==  {'precision': [0.8334777355194092], 'recall': [0.8810116052627563], 'f1': [0.856585681438446], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.50.3)'}


Raw out: answe

In [182]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["An SFU computing ID"]
references = ["An SFU Computing ID"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print("Results == ", results)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results ==  {'precision': [0.9849278926849365], 'recall': [0.9849278926849365], 'f1': [0.9849278926849365], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.50.3)'}
